# <span style='color:red'>Project 1</span>

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

/var/folders/b4/jh08kpys74l03tzk4ffbltkh0000gn/T/ipykernel_1347/1189900070.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#### In this project we use dynamic programming to create a trading schedule that maximizes total number of shares traded, under a model of liquidity impact with memory

In [ ]:
import csv
import sys
import scipy.io
import numpy as np
import math
import matplotlib.pyplot as plt

# #### Suppose we have a total of N shares that we would like to trade over T time periods.  To do so, we produce a schedule
$$ (n_0, n_1, \ldots, n_{T-1}) \quad \text{where each} \quad n_i \ge 0$$
#### Each $n_i$ represents the quantity that we will attempt  to trade at time $i = 0, 2, \ldots, T-1$.  In reality the market will only allow us to trade a smaller quantity at each time period.  We impose the conditions:
$$ \sum_{i=0}^{T-2} n_i \ \le N \quad \text{and} \quad n_{T-1} = N - \text{quantity traded so far}$$
#### This plays out as follows.  Assume that $\alpha > 0$ (and very small) and $0 < \pi < 1$ are given parameters.  Then we run the following process:
#### 1. Initialize $M = 0$.  Then for $i = 0, 2, \ldots, T-1$ we do the following:
#### 2. Compute $M \leftarrow \lceil 0.1*M + 0.9*n_i\rceil$.
#### 3. At time $i \le T-1$ we trade $S_i \ = \ \lceil(1 - \alpha M^\pi)n_i \rceil$ shares.  
#### 4. Note that $n_{T-1} = N \, - \, \sum_{i = 0}^{T-2} n_i$.

#### <span style='color:red'>Example:  N = 10000, T = 4,   $\alpha = 0.001$,   $\pi = 0.5$</span>


In [ ]:
M = 0
T = 10
N = 100000
alpha = 1e-3
pi = 0.5
S = np.zeros(T,dtype='i')
n  = np.array([ 9540.,  9750.,  9874.,  9901.,  9886.,  9851.,  9741.,  9793.,
       10471., 11193.]) # note that we index the array starting from zero
print("Planned sales schedule:",n,"; total planned = ",np.sum(n))
total = 0
for i in range(T):
    M = math.ceil(0.1*M + 0.9*n[i])
    S[i] = math.ceil((1 - alpha*M**pi)*n[i])
    total += S[i]
    print('at time %d, M = %d and we trade %d shares' %(i,M,S[i]))
print('total sold =', total, "i.e., as a percentage,",100*np.sum(S)/np.sum(n),"of the total planned.")

Planned sales schedule: [ 9540.  9750.  9874.  9901.  9886.  9851.  9741.  9793. 10471. 11193.] ; total planned =  100000.0
at time 0, M = 8586 and we trade 8657 shares
at time 1, M = 9634 and we trade 8794 shares
at time 2, M = 9850 and we trade 8895 shares
at time 3, M = 9896 and we trade 8917 shares
at time 4, M = 9887 and we trade 8904 shares
at time 5, M = 9855 and we trade 8874 shares
at time 6, M = 9753 and we trade 8780 shares
at time 7, M = 9789 and we trade 8825 shares
at time 8, M = 10403 and we trade 9404 shares
at time 9, M = 11114 and we trade 10014 shares
total sold = 90064 i.e., as a percentage, 90.064 of the total planned.


In [ ]:
M = 0
T = 10
N = 100000
alpha = 1e-3
pi = 0.5
S = np.zeros(T,dtype='i')
n  = np.ones(T)*(N/T) # note that we index the array starting from zero
print("Planned sales schedule:",n,"; total planned = ",np.sum(n))
total = 0
for i in range(T):
    M = math.ceil(0.1*M + 0.9*n[i])
    S[i] = math.ceil((1 - alpha*M**pi)*n[i])
    total += S[i]
    print('at time %d, M = %d and we trade %d shares' %(i,M,S[i]))
print('total sold =', total, "i.e., as a percentage,",100*np.sum(S)/np.sum(n),"of the total planned.")

Planned sales schedule: [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.] ; total planned =  100000.0
at time 0, M = 9000 and we trade 9052 shares
at time 1, M = 9900 and we trade 9006 shares
at time 2, M = 9990 and we trade 9001 shares
at time 3, M = 9999 and we trade 9001 shares
at time 4, M = 10000 and we trade 9000 shares
at time 5, M = 10000 and we trade 9000 shares
at time 6, M = 10000 and we trade 9000 shares
at time 7, M = 10000 and we trade 9000 shares
at time 8, M = 10000 and we trade 9000 shares
at time 9, M = 10000 and we trade 9000 shares
total sold = 90060 i.e., as a percentage, 90.06 of the total planned.


### <span style='color:red'>Task 1: </span>code a dynamic programming algorithm that computes an optimal schedule of trades $(n_0, n_1, \ldots, n_{T-1})$ with the goal of maximizing the total number of traded shares
#### Make sure that your code runs well for a range of values of $\alpha$ and $\pi$
#### Compute the optimal schedule when $\alpha = 0.001$, $\pi = 0.5$, $N = 100000$ and $T = 10$.   Denote this schedule by $(S_0, S_1, \ldots, S_9)$.

In [ ]:
def create_momentum(M, n_arr):
    #Helper function
    #Calculate an array of momentum based on the array of previous momentum and schedule
    return np.ceil(np.add(0.1*M,0.9*n_arr))

In [ ]:
def sell_given_m_n(M, n_arr):
    #Helper function
    #Determine the number of shares you are able to sell based on an array of momenum and schedule
    return np.ceil((1 - alpha*M**pi)*n_arr)

In [ ]:
def DP1(N, T, alpha, pi):
  #Dynamic Programming to determine the optimal schedule for each time period.
  # DP optimal tables
  # M[t, j] is momentum at time t, assuming j shares are scheduled optimally from time 0 to time t.
  M = np.full((T, N+1), -np.inf)
  # S[t, j] is the optimal number of total shares sold from time 0 to time t, assuming j shares are scheduled optimally from time 0 to time t.
  S = np.full((T, N+1), -np.inf)
  # decision[t, j] is number of shares scheduled at time t, assuming j shares are scheduled optimally from time 0 to time t.
  decision = np.full((T, N+1), -np.inf)

  # enumeration arrays
  iarray = np.array(range(N + 1)) # helper array [0,1,2,3,...,N]
  revi = np.array(range(N, -1, -1)) # helper array [N,N-1,N-2,...,0]

  # candidates
  M_candidates = np.full((N+1), -np.inf) #All the candidate M at time i.
  S_candidates = np.full((N+1), -np.inf) #All the candidate S at time i.

  # The base case is at time 0 without any previous momentum
  # The momentum you create at time 0 is simply 0.9 * the shares you schedule. We enumerate all schedules from 0 to N.
  np.copyto(M[0, :], np.ceil(0.9*iarray))
  # The number of shares you sold at time 0 based on M and our formula.
  np.copyto(S[0, :], np.ceil((1 - alpha*M[0, :]**pi)*iarray))
  # Your corresponding schedule at time 0, which is simply an enumeration from 0 to N.
  np.copyto(decision[0, :], range(N + 1))

  # Dynamic Programming Recurrence.
  # Iterate over time t=1,2,...,T-1.
  for i in range(1, T):
      # Iterate over j=0,1,2,...,N shares total scheduled optimally from time 0 to time t.
      for j in range(0, N + 1):
          # for k = 0,1,2,...j, calculate M_candidates[k], using M[t-1, k] and the j-k shares you schedule at time t.
          np.copyto(M_candidates[:j + 1], create_momentum(M[i - 1, :j + 1],revi[N - j:N + 1]))

          # for k = 0,1,2,...j, calculate S_candidates[k], using M_candidates[k], S[t-1,k] and the j-k shares you schedule at time t.
          np.copyto(S_candidates[:j + 1], np.add(S[i - 1, :j + 1], sell_given_m_n(M_candidates[:j + 1],revi[N - j:N + 1])))

          #find max{S_candidates[k]}, for k = 0,1,2,...j, and store the value.
          S[i, j] = np.max(S_candidates[:j + 1])
          # find k*= argmax{S_candidates[k]}, for k = 0,1,2,...j
          # find M_candidates[k*] and store the value.
          M[i, j] = M_candidates[np.argmax(S_candidates[:j + 1])]
          # find j-k* and store the value
          decision[i, j] = revi[N - j:N + 1][np.argmax(S_candidates[:j + 1])]

  # Back out the schedule
  # Store the schedule
  schedule = np.zeros(T)
  # keeps track of shares unsold so far.
  total_unsold = N
  # Working from backward (starting time T-1) because S[T-1, N] is the terminal states, telling us the total performance.
  for t in range(T - 1, -1, -1):
      # Retrieve the decision.
      schedule[t]=int(decision[t, total_unsold])
      # Reduce the total_unsold by the decision.
      total_unsold -= int(decision[t, total_unsold])

  return schedule

In [ ]:
DP1(100000, 10, 0.001, 0.5)

array([ 9540.,  9750.,  9874.,  9901.,  9886.,  9851.,  9741.,  9793.,
       10471., 11193.])

We test the performance of our schedule in the example section. It is better than selling 10,000 shares at each period.

Planned sales schedule: [ 9540.  9750.  9874.  9901.  9886.  9851.  9741.  9793. 10471. 11193.] ; total planned =  100000.0

at time 0, M = 8586 and we trade 8657 shares

at time 1, M = 9634 and we trade 8794 shares

at time 2, M = 9850 and we trade 8895 shares

at time 3, M = 9896 and we trade 8917 shares

at time 4, M = 9887 and we trade 8904 shares

at time 5, M = 9855 and we trade 8874 shares

at time 6, M = 9753 and we trade 8780 shares

at time 7, M = 9789 and we trade 8825 shares

at time 8, M = 10403 and we trade 9404 shares

at time 9, M = 11114 and we trade 10014 shares

total sold = 90064 i.e., as a percentage, 90.064 of the total planned.

Our schedule:

[ 9540, 9750, 9874, 9901, 9886, 9851, 9741, 9793, 10471, 11193]



### <span style='color:red'>Task 2. Test the effectiveness of this computed schedule using the first 2 hours of each day in the TSLA data </span>
To do so, we divide the first 2 hours of each day into 12 separate intervals of ten minutes each.
Each interval is evaluated as follows.  Suppose that the traded volume in that interval is given by the numbers $(V_0, V_1, \ldots, V_9)$.
Then the interval score we assign to our schedule is given by
$$ \sum_{i = 0}^9 \min\{ S_i, V_i/100 \}.$$
Effectively, this scheme allows us to trade up to a volume of 1% of what the market actually traded.

#### The TOTAL SCORE we assign to our schedule is the average of the all interval scores, averaged over the first 12 intervals of all the days in the first half of our data
#### In other words, if we have 300 days of data, we take the first 150, and we get in total 12x150 = 1800 intervals

In [ ]:
filename = 'TSLA.csv'
f = open(filename,'r')
csvf = csv.reader(f)
thelist = list(csvf)
f.close()
numobservations = len(thelist)
#print('total number of rows:', numobservations)
nlist = np.array(thelist)
stringvolume = nlist[4:51630,6] #column 6 (from 0) has the trading volume per minute
#print('first ten entries of column 6:',stringvolume[:10])
#convert to integers
#let's try to guess what the problem is
numobservations = len(thelist)
#print('total number of rows:', numobservations)
nlist = np.array(thelist)
stringvolume = nlist[4:,6]
# One solution.  Most efficient? Find the length of each entry and use that to find the first empty one.
# Well, it works.
length_checker = np.vectorize(len)

v_len = length_checker(stringvolume)
#v_len will be an array whose kth entry is the length of the string in position k in stringvolume
#print('size of some selected entries:',v_len[51620:51630])

first0 = np.argmin(v_len)
#print('first blank at', first0)
#print('check:', v_len[first0-1], v_len[first0])
numactual = first0
#and convert to integers

volume = stringvolume[:numactual].astype(int) #in other words I am taking all entries up to the first blank one
#check
#print('first', volume[0], 'last', volume[numactual-1])
#plt.plot(volume)

In [ ]:
nlist = np.array(thelist)
stringdt = nlist[4:51630,0] #column 6 (from 0) has the trading volume per minute
#convert to datetime
from datetime import datetime
from dateutil import parser
dt = np.zeros(numactual,dtype='datetime64[s]')
for i in range(numactual):
    dt[i] = parser.parse(stringdt[i])
#check
#print('first', dt[0], 'last', dt[numactual-1])
#how many days of data do we have?
numdays = (dt[numactual-1]-dt[0])/ np.timedelta64(1, 'D')
#print('number of days of data:', numdays)

In [ ]:
#extract first 95 days
firstday = dt[0]
lastday = firstday + np.timedelta64(96, 'D')
mask = (dt >= firstday) & (dt <= lastday)
dt = dt[mask]
#check
print('first', dt[0], 'last', dt[-1])
#how many days of data do we have?
numdays = (dt[-1]-dt[0])/ np.timedelta64(1, 'D')
#print('number of days of data:', numdays)
dtlen = dt.size
#print(dtlen)

first 2021-01-04T09:30:00 last 2021-04-09T16:29:00


In [ ]:
#find index of each openning
idx = np.concatenate([np.where(dt == dt[0] + a * np.timedelta64(1, 'D'))[0] for a in range(96) ])
#idx

In [ ]:
#compute volume of each interval
itvVolume = []

for i in idx:
    cr = i
    for j in range(12):
        itvVolume.append(np.sum(volume[cr:cr+10]))
        cr = cr + 10
itvVolume = np.array(itvVolume)
#plt.plot(itvVolume)

In [ ]:
intervals = []

for i in idx:
    interval_data = [volume[i+j:i+j+10] for j in range(0, 120, 10)]  # Extract 10-minute intervals
    intervals.extend(interval_data)
intervals = np.array(intervals)
#print(intervals)

In [ ]:
S = np.array([ 9540.,  9750.,  9874.,  9901.,  9886.,  9851.,  9741.,  9793.,
       10471., 11193.])

In [ ]:
def backtest(intervals, itvVolume, S):
    # s is the schedule
    # s = function generating schedule
    # itvVolume is the total volume per 10 minutes (N)
    #inrervals is the volume per minute (V_i)
    # returns the score
    score = 0
    for i in range(itvVolume.size):
        # s is the schedule
        # s = function generating schedule with input itvVolume[i]
        itvScore = 0
        for j in range(10):
            itvScore += min(S[j], intervals[i][j]/100)
        score += itvScore

    score = score/itvVolume.size
    return score

In [ ]:
backtest(intervals, itvVolume, S)

8059.231330845776

The score for our schedule is 8059.231330845776

### <span style='color:red'>Task 3:</span>  code an algorithm that (approximately) does the following:
#### 1. It approximately enumerates all possible values for $\pi$ between $0.3$ and $0.7$
#### 2. It approximately computes the value of $\pi$ that maximizes the TOTAL SCORE, when $N = 100000$, $T = 10$ and $\alpha = 0.001$.
#### 3. This means that we run the DP algorithm (under the chosen value of $\pi$) and then evaluate as above to compute the TOTAL SCORE.

In [ ]:
# Define the parameters
N = 100000
T = 10
alpha = 0.001

# Define the range of π values to consider
pi_values = np.linspace(0.3, 0.7, 5)  # Adjust the number of values as needed

# Initialize variables to store the best result
best_pi = None
best_pi_score = float("-inf")

for pi in pi_values:
    print(pi)
    #utilize DP algorithm from part 1 to calculate optimal schedule given parameters N, T, alpha, and pi
    pi_schedule = DP1(N, T, alpha, pi)
    #utilize score calculation function which simply draws data from TSLA.csv and draws first 2 hours of each day to calculate
    #effectiveness of a given schedule
    pi_score = backtest(intervals, itvVolume, pi_schedule)
     # Check if this π value gives a better score, if it does then update the optimal pi value
    if pi_score > best_pi_score:
        best_pi_score = pi_score
        best_pi = pi
# Print the optimal pi as well as the score of the corresponding schedule
print(f"Best π value: {best_pi}")
print(f"Best SCORE: {best_pi_score}")

0.3
0.39999999999999997
0.5
0.6
0.7
Best π value: 0.7
Best SCORE: 99932.26990049751
